In [300]:
# Reload modules automatically on changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Required import

In [301]:
import tsdf
import os
import sys
import numpy as np

## Process an existing binary file and write the new TSDF metadata
Read and process an existing binary data (accompanied by the TSDF metadata), process the data and save it in the new format, with the corresponding TSDF metadata file.

### Load dummy data and see its format

In [302]:
TESTDATA_DIR = os.path.join("..","tests", "data")
TESTOUT_DIR = os.path.join(TESTDATA_DIR, "local_test_out")
dummy_data_name = "dummy_10_3_int16"

dummy_metadata = tsdf.load_metadata_from_path(os.path.join(TESTDATA_DIR, dummy_data_name + ".json"))[dummy_data_name + ".bin"]
dummy_data = example_1_metadata.load_binary()
print(f"Data type used for storing:\t {dummy_data.dtype}")
print(f"Data dimensions:\t\t {dummy_data.shape}")
print(f"Number of rows:\t\t\t {dummy_data.shape[0]}")

NameError: name 'example_1_metadata' is not defined

### Perform light data processing

In [ ]:
processed_dummy_data_1 = (dummy_data / 10).astype('float32')
print(f"Data type used for storing:\t {processed_dummy_data_1.dtype}")
print(f"Data dimensions:\t\t {processed_dummy_data_1.shape}")
print(f"Number of rows:\t\t\t {processed_dummy_data_1.shape[0]}")

Data type used for storing:	 float32
Data dimensions:		 (10, 3)
Number of rows:			 10


### Metadata available from the data (NumPy array)
The metadata will be used indirectly to generate the new TSDF metadata file.

In [ ]:
bin_meta = tsdf.get_metadata_from_ndarray(processed_dummy_data_1)
print(f"Binary formatting that can be inferred from the NumPy array:\n{bin_meta}")

Binary formatting that can be inferred from the NumPy array:
{'data_type': 'float', 'bits': 32, 'endianness': 'little', 'rows': 10}


### Write the processed data 
Write the processed data in binary format. The call returns the corresponding metadata object.

In [ ]:
processed_dummy_data_name_1 = "dummy_10_3_int16_to_float32"
processed_dummy_metadata_1 = tsdf.write_binary_file(
            TESTOUT_DIR,
            processed_dummy_data_name_1 + ".bin",
            processed_dummy_data_1,
            dummy_metadata.get_plain_tsdf_dict_copy(),
        )

### Write the TSDF metadata file that describes the processed binary data format

#### 1) Write the metadata file for a single binary file

In [ ]:
# Write new metadata file
tsdf.write_metadata([processed_dummy_metadata_1], processed_dummy_data_name_1 + ".json")


#### 2) Write a metadata file that combines multiple binary files

In [ ]:
# Preprocess the original data to generate another data source
processed_dummy_data_2 = (dummy_data * 1000).astype('int32')

# Adjust the metadata slightly
updated_dummy_metadata = dummy_metadata.get_plain_tsdf_dict_copy()
updated_dummy_metadata.pop("scale_factors") #remove the 'scale_factors'


# Save the new binary file
processed_dummy_data_name_2 = "dummy_10_3_int16_to_int32"
processed_dummy_metadata_2 = tsdf.write_binary_file(
            TESTOUT_DIR,
            processed_dummy_data_name_2 + ".bin",
            processed_dummy_data_2,
            updated_dummy_metadata,
        )

# Write a metadata file that combines the two binary files
tsdf.write_metadata([processed_dummy_metadata_1, processed_dummy_metadata_2], "dummy_10_3_int16_to_int_n_float.json")


## Genarate a new binary file and the corresponding TSDF metadata
Generate binary data and save it and the corresponding TSDF metadata file.

In [303]:
path = os.path.join(TESTDATA_DIR, "test_output_1.bin")
rs = np.random.RandomState(seed=42)
data_1 = rs.rand(17, 1).astype(np.float32)
data_2 = rs.rand(15, 2).astype(np.int16)

#new_meta_2 =  tsdf.write_binary_file(TESTOUT_DIR, "test_2.bin", data_2, loaded_meta.get_plain_tsdf_dict_copy())

# An example where the metadata is defined from scratch
new_metadata = {}
new_metadata["subject_id"] = "example"
new_metadata["study_id"] = "example"
new_metadata["device_id"] = "example"
new_metadata["endianness"] = "little"
new_metadata["metadata_version"] = "0.1"
new_metadata["start_datetime_unix_ms"] = 1571135957025,
new_metadata["start_iso8601"] = "2019-10-15T10:39:17.025000+00:00"
new_metadata["end_datetime_unix_ms"] = 1571168851826
new_metadata["end_iso8601"] = "2019-10-15T19:47:31.826000+00:00"
new_metadata["channels"] = ["x","y","z"]
new_metadata["units"] = ["m/s/s","m/s/s","m/s/s"]

# Write the two binary files based on the provided metadata

new_meta_1 =  tsdf.write_binary_file(TESTOUT_DIR, "test_1.bin", data_1, new_metadata)
new_meta_2 =  tsdf.write_binary_file(TESTOUT_DIR, "test_2.bin", data_2, new_metadata)


Save the metadata that corresponds to the binary data. In case of multiple binary files, the corresponding metadata files have to be combined.

In [304]:
# Write the first metadata file
tsdf.write_metadata([new_meta_1], "test1.json")

In [305]:
# Combine and write both metadata files
tsdf.write_metadata([new_meta_1, new_meta_2], "test2.json")